In [2]:
import os
import sys
import xml.etree.ElementTree as ET

import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
sys.path.insert(0, '../src')
import dm

In [8]:
settings_xml = dm._registry.get_ba_key_value('SettingsFile')
bds_dir = dm._registry.get_ba_key_value('DemographyDataDir1')

tree = ET.parse(settings_xml)
root = tree.getroot()

x_pth_lyrs = './/TASK[@id="GeographyLevelCategory.StandartGeographyTA"]'\
             '//ARRAY_PROPERTY[@name="GeographyLevelLayers"]'

# get the parent element for all layers
e_lyrs = root.find(x_pth_lyrs)

# get the parent element for each respective layer
e_lst_lyrs = e_lyrs.findall('COMPLEX_PROPERTY')

def _get_path_from_bds(bds_pth):
    bds_tree = ET.parse(bds_pth)
    bds_root = bds_tree.getroot()

    e_src = bds_root.find('.//MappingLayer/DataSource')
    gdb = e_src.find('WS').attrib['PathName'].replace('.\\', '')
    fds = e_src.find('FeatureDataset').attrib['FeatureDataset']
    fc = e_src.find('Dataset').attrib['Dataset']

    return os.path.join(bds_dir, gdb, fds, fc)

def _get_lyr(e_lyr):
    alias = e_lyr.find('./SIMPLE_PROPERTY[@name="Caption"]').attrib['value']
    name = alias.lower().replace(' ', '_').replace('(', '').replace(')', '')
    
    e_fl = e_lyr.find('./COMPLEX_PROPERTY[@name="DataFeatureLayer"]')
    
    if e_fl:
        col_id = e_fl.find('./SIMPLE_PROPERTY[@name="IDField"]').attrib['value']
        col_name = e_fl.find('./SIMPLE_PROPERTY[@name="NameField"]').attrib['value']
        
        bds_lyr_nm = e_fl.find('./ARRAY_PROPERTY[@name="DetalizationLayers"]/COMPLEX_PROPERTY/SIMPLE_PROPERTY[@name="LevelName"]').attrib['value']
        bds_pth = os.path.join(bds_dir, bds_lyr_nm)
        
        fc_pth = _get_path_from_bds(bds_pth)
        
    else:
        col_id, col_name, fc_pth = None, None, None
    
    return (name, alias, col_id, col_name, fc_pth)

row_lst = [_get_lyr(e) for e in e_lst_lyrs]

df = pd.DataFrame(row_lst, columns=['name', 'alias', 'col_id', 'col_name', 'feature_class_path']).dropna()

df.iloc[::-1].reset_index(drop=True)

,name,alias,col_id,col_name,feature_class_path
0,block_groups,Block Groups,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
1,census_tracts,Census Tracts,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
2,cities_and_towns_places,Cities and Towns (Places),ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
3,zip_codes,ZIP Codes,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
4,county_subdivisions,County Subdivisions,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
5,counties,Counties,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
6,cbsas,CBSAs,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
7,congressional_districts,Congressional Districts,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
8,dmas,DMAs,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
9,states,States,ID,NAME,D:\arcgis\ba_data\Data\Demographic Data\USA_ES...
